In [3]:
import numpy as np
import pandas as pd
import jieba
import pyLDAvis
from biterm.btm import oBTM 
from sklearn.feature_extraction.text import CountVectorizer
from biterm.utility import vec_to_biterms, topic_summuary # helper functions
import re

# 待做 LDA 的文本 csv 文件，可以是本地文件，也可以是远程文件，一定要保证它是存在的！！！！
# source_csv_path = 'answers.csv'
source_csv_path = './test.csv'
# 文本 csv 文件里面文本所处的列名,注意这里一定要填对，要不然会报错的！！！
# document_column_name = '回答内容'
document_column_name = 'reviewinfo'
# 输出主题词的文件路径
top_words_csv_path = 'btm-top-topic-words.csv'
# 输出各文档所属主题的文件路径
predict_topic_csv_path = 'btm-document-distribution.csv'
# 可视化 html 文件路径
html_path = 'btm-document-visualization.html'
# 选定的主题数
n_topics = 3
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words = 20
# 去除无意义字符的正则表达式
pattern = u'[\\s\\d,.<>/?:;\'\"[\\]{}()\\|~!\t"@#$%^&*\\-_=+，。\n《》、？：；“”‘’｛｝【】（）…￥！—┄－]+'

df = (
    pd.read_csv(
        source_csv_path,
        encoding='utf-8-sig')
    .drop_duplicates()
    .rename(columns={
        document_column_name: 'text'
    }))
# 设置停用词集合
stop_words_set = set(['你', '我'])
# 去重、去缺失、分词
df['cut'] = (
    df['text']
    .apply(lambda x: str(x))
    .apply(lambda x: re.sub(pattern, ' ', x))
    .apply(lambda x: " ".join([word for word in jieba.lcut(x) if word not in stop_words_set]))
)

texts = list(df['cut'].values)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.786 seconds.
Prefix dict has been built successfully.


In [7]:
# vectorize texts
stop_words_list = ['你', '我']
vec = CountVectorizer(stop_words=stop_words_list, dtype=np.float32)
X = vec.fit_transform(texts).toarray()

# get vocabulary
vocab = np.array(vec.get_feature_names())

# get biterms
biterms = vec_to_biterms(X)

# create btm
btm = oBTM(num_topics=5, V=vocab)

print("\n\n Train Online BTM ..")
for i in range(0, len(biterms), 100): # prozess chunk of 200 texts
    biterms_chunk = biterms[i:i + 100]
    btm.fit(biterms_chunk, iterations=5)
topics = btm.transform(biterms)

print("\n\n Visualize Topics ..")
vis = pyLDAvis.prepare(btm.phi_wz.T, topics, np.count_nonzero(X, axis=1), vocab, np.sum(X, axis=0))
pyLDAvis.save_html(vis, './vis/online_btm.html')

print("\n\n Topic coherence ..")
res = topic_summuary(btm.phi_wz.T, X, vocab, 10)
with open(file='btm-top-words.txt', mode='w+') as file:
    for z in range(len(res['coherence'])):
        file.writelines('Topic {} | Coherence={:0.2f} | Top words= {}\n'.format(z, res['coherence'][z], ' '.join(res['top_words'][z])))

print("\n\n Texts & Topics ..")
with open(file='btm-topic-of-text.txt', mode='w+') as file:
    for i in range(len(texts)):
        file.writelines("(topic: {}) {}\n".format(topics[i].argmax(), texts[i]))

/home/albert/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)




 Train Online BTM ..


100%|██████████| 5/5 [01:10<00:00, 14.12s/it]




 Visualize Topics ..


 Topic coherence ..
Topic 0 | Coherence=-41.25 | Top words= 辣酱 圈子 八宝 草头 没有 有点 比较 特别 不错 老字号
Topic 1 | Coherence=-34.88 | Top words= 告知 圈子 份量 草头 大肠 食客 本来 特地去 点评 一桌
Topic 2 | Coherence=-8.77 | Top words= 传统 跟不上 排队 砂锅 三鲜 冰糖 就是 纯正 漫长 信号
Topic 3 | Coherence=-62.98 | Top words= 不错 上海 本帮菜 味道 口味 好吃 非常 环境 正兴 不是
Topic 4 | Coherence=-50.77 | Top words= 服务员 饭店 上海 服务 就是 国营 赤酱 一个 浓油 顾客


 Texts & Topics ..
顶 着 老字号 的 招牌 味道 实在 一般 除了 交通 方便 其他 找不出 能 美言 的 地方 这 就是 老牌 不思进取 慢慢 衰退 (topic: 0)
很 上海 的 菜肴   就是 份量 偏少   菜肴 口味 很 好   (topic: 2)
很 不错   蛮 有 风味 的   下次 还会 再 来 (topic: 1)
老 正兴 八宝 辣酱 粽   做 为 本 帮菜 的 老字号 八宝 辣酱 一直 是 的 最 爱 吃 久 了 蛋黄 粽来 吃款 新品 吧 偶然 路过 福州路 发现 老 正兴 推出 了 八宝 辣酱 粽   立马 长草 了 八宝 的 酱汁 与 糯米 的 混合 入口 咸中 带鲜 八宝 君   鸭 胗 鸡胸肉 猪肚 茭白 香菇 豆腐干 肥瘦 相间   (topic: 0)
八宝 辣酱 粽 真是 惊艳 到 啦   (topic: 0)
草头 圈子 味道 一级 棒   只是 圈子 好少 大概 六块 吧   蟹粉 狮子头 真是太 让 人 失望 了   一口 的 面粉 没 吃 出蟹 味 也 没有 肉味 呀   荠菜 馄饨 还是 不错 的   总体而言 价格 偏高 味道 有些 小 遗憾 吧   (topic: 3)
正兴 酱方 整体 不错   偏甜 不腻   喜好 甜食 的 有 福 了   油爆 虾 过得去   草头 比较 嫩   但 有 一口

/home/albert/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
